In [1]:
from bs4 import BeautifulSoup, Comment
import os
import sys
import re
sys.path.append('..')
from dateestimation import DateEstimation 
from tools.envvars import load_env_vars_from_directory

In [2]:
load_env_vars_from_directory('../.env')
cal_file =os.getenv('PODCAST_CAL_FILE')
if not cal_file:
    raise ValueError("PODCAST_CAL_FILE environment variable is not set.")
if not os.path.exists(cal_file):
    raise FileNotFoundError(f"Calendar '{cal_file}' does not exist.")

prefix = os.getenv('PODCAST_PREFIX')
if not prefix:
    raise ValueError("PODCAST_PREFIX environment variable is not set.")

templates = os.getenv('PODCAST_TEMPLATES', '../templates')
css = os.path.join(templates, 'podcast.css')
if not os.path.exists(css):
    raise FileNotFoundError(f"CSS file '{css}' does not exist.")

workdir = os.getenv('PODCAST_WORKDIR', '../workdir')
if not os.path.exists(workdir):
    raise FileNotFoundError(f"Work directory '{workdir}' does not exist.")

print(f"Using calendar file: {cal_file}")
print(f"Using podcast prefix: {prefix}")
print(f"Using templates directory: {templates}")
print(f"Using work directory: {workdir}")
print(f"Using CSS file: {css}")

Using calendar file: /home/jmrobles/Podcasts/Cowboys de Medianoche/DB/cmcal.csv
Using podcast prefix: cm
Using templates directory: /home/jmrobles/Podcasts/Cowboys de Medianoche/templates
Using work directory: /home/jmrobles/Podcasts/Cowboys de Medianoche/
Using CSS file: /home/jmrobles/Podcasts/Cowboys de Medianoche/templates/podcast.css


In [3]:
tdir = workdir
!ls "$tdir"

addtocal.bash			cm251219_whisper_es.srt
Archivo				DB
Authorized			Pendiente
Barroco				Sttcast
cm251219.meta			Summaries
cm251219.mp3			templates
cm251219_vosk_audio_es.html	Tmp
cm251219_vosk_es.html		ToSummarize
cm251219_whisper_audio_en.html	training.meta
cm251219_whisper_audio_es.html	training.mp3
cm251219_whisper_en.html	training.mp3.gen
cm251219_whisper_en.srt		training.mp3.simple
cm251219_whisper_es.html	validate.bash


In [4]:
def get_epnumber(filename):
    # cálculo de basename del archivo
    basename = os.path.basename(filename)
    
    get_epnumber = re.compile(rf'{prefix}([^\.]+).*')
    match = get_epnumber.search(basename)
    if match:
        epnumber = match.group(1)
        # Remover las partes relacionadas con vosk y whisper
        epnumber = re.sub(r'_vosk[^\.]*', '', epnumber)
        epnumber = re.sub(r'_whisper[^\.]*', '', epnumber)
    return epnumber

In [5]:
print(get_epnumber(f'{prefix}519_B_vosk.mp3'))
print(get_epnumber(f'{prefix}250731_en_whisper.mp3'))
print(get_epnumber(f'{prefix}250731.mp3'))

519_B
250731_en
250731


In [6]:
import glob
mp3_files = glob.glob(os.path.join(tdir, f'{prefix}*.mp3'))
print (mp3_files[:10])

['/home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219.mp3']


In [7]:
import pysrt

In [8]:
textpat = re.compile(r'^\[(?P<speaker>.*)\]: *(?P<spoken>.*)')

In [9]:
def change_speaker_in_srt_file(srt_file, old_speaker, new_speaker):
    subs = pysrt.open(srt_file, encoding='utf-8')
    for sub in subs:
        if sub.text.startswith(f"[{old_speaker}]:"):
            sub.text = sub.text.replace(f"[{old_speaker}]:", f"[{new_speaker}]:")
    subs.save(srt_file, encoding='utf-8')


In [10]:
change_dict = {    

   '251114': {
      'Luia Enríquez': 'Luis Enríquez',

   },

   '251128': {
      'Luis Herrero0': 'Unknown 20',

   },
   '251205': {
      'Luis Herrero0': 'Unknown 20',

   },
   '251219': {
      'Noemi Guillermo': 'Unknown 20',
      'Unknown 2': 'Luis Herrero',
      'Unknown 3': 'José Luis Garci',
      'Unknown 4': 'Eduardo Torres-Dulce',
      'Unknown 5': 'Luis Alberto de Cuenca',
      'Unknown 6': 'Inocencio Arias',
   },
}

In [11]:
for mp3f in mp3_files:
    epnumber = get_epnumber(mp3f)
    if epnumber is None:
        print(f"Could not extract episode number from {mp3f}")
        continue
    print (f"Processing episode number: {epnumber}")
    if epnumber not in change_dict:
        print(f"No changes defined for episode number {epnumber}, skipping.")
        continue
    for strf in glob.glob(os.path.join(tdir, f'{prefix}{epnumber}_*.srt')):
        print(f"Processing subtitle file: {strf}")
        for old_speaker, new_speaker in change_dict[epnumber].items():
            print(f"Changing speaker from '{old_speaker}' to '{new_speaker}' in {strf}")
            change_speaker_in_srt_file(strf, old_speaker, new_speaker)


Processing episode number: 251219
Processing subtitle file: /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Noemi Guillermo' to 'Unknown 20' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Unknown 2' to 'Luis Herrero' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Unknown 3' to 'José Luis Garci' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Unknown 4' to 'Eduardo Torres-Dulce' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Unknown 5' to 'Luis Alberto de Cuenca' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Changing speaker from 'Unknown 6' to 'Inocencio Arias' in /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_en.srt
Processing subtitle file: /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_es.

In [12]:
html_whisper_files = glob.glob(os.path.join(tdir, f'{prefix}*_whisper*.html'))
for htmlf in html_whisper_files:
    epnumber = get_epnumber(htmlf)
    if epnumber is None:
        print(f"Could not extract episode number from {mp3f}")
        continue
    if epnumber not in change_dict:
        print(f"No changes defined for episode number {epnumber}, skipping.")
        continue
    print(f"Processing HTML file: {htmlf}")
    with open(htmlf, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
    # Localizar todos los textos de class "speaker*"
    speakers = soup.find_all(class_=re.compile(r'speaker-\d+'))
    # Sustituir, si existe old_speaker por new_speaker
    olds = []
    news = []
    for old_speaker, new_speaker in change_dict.get(epnumber, {}).items():
        print(f"Changing speaker from '{old_speaker}' to '{new_speaker}' in HTML file {htmlf}")
        for speaker in speakers:
            # Cambiar el texto old_speaker for new_speaker dejando el resto del texto intacto
            if old_speaker in speaker.text:
                speaker.string = speaker.text.replace(old_speaker, new_speaker)
        olds.append(old_speaker)
        news.append(new_speaker)
    # Localizar el texto <!-- old_speaker ha hablado ... y cambiar por <!-- new_speaker ha hablado ... -->
    for oldsp in olds:
        newsp = news[olds.index(oldsp)]
        print(f"Changing comment from '{oldsp}' to '{newsp}' in {htmlf}")
        comments = soup.find_all(
            string=lambda text: isinstance(text, Comment)
                                and f"{oldsp} ha hablado" in text
        )
        if not comments:
            print(f"No comments found for speaker '{oldsp}' in {htmlf}")
        for comment in comments:
            print (f"Changing comment from '{comment}' to include speaker '{newsp}'")
            new_text = comment.replace(oldsp, newsp)
            comment.replace_with(Comment(new_text))

    #Localizar el span "speaker_summary" y dentro de el una lista ul. Se borrará y rellenará con los hablantes
    # tomados de los comentarios
    speaker_summary = soup.find('span', id='speaker-summary')
    if speaker_summary:
        ul = speaker_summary.find('ul')
        if ul:
            # Borrar los elementos de la lista
            ul.clear()
        else:
            print(f"No 'ul' found in 'speaker-summary' span in {htmlf}")
    else:
        print(f"No 'speaker-summary' span found in {htmlf}")

    # Si no existe, crear un nuevo ul    
    # Una vez cambiados algunos comentarios vamos a repasar todos los comentarios y 
    # componer la lista de hablantes
    comments = soup.find_all(
    string=lambda text: isinstance(text, Comment)
                        and f" ha hablado" in text
    )   
    for comment in comments:
        # Extraer el hablante del comentario
        match = re.search(r'(.*?) ha hablado (.*?) en el segmento', comment)
        if match:
            speaker_name = match.group(1).strip()
            speaker_time = match.group(2).strip()
            # Si speaker_name empieza por "Unknown" o por "?" no lo añadimos
            if speaker_name.startswith('Unknown') or speaker_name.startswith('?'):
                continue
            li = soup.new_tag('li')
            li.string = f"{speaker_name}: {speaker_time}"
            ul.append(li)
                                    
    
    # Guardar el archivo modificado
    with open(htmlf, 'w', encoding='utf-8') as f:
        f.write(str(soup))



Processing HTML file: /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Noemi Guillermo' to 'Unknown 20' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Unknown 2' to 'Luis Herrero' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Unknown 3' to 'José Luis Garci' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Unknown 4' to 'Eduardo Torres-Dulce' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Unknown 5' to 'Luis Alberto de Cuenca' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing speaker from 'Unknown 6' to 'Inocencio Arias' in HTML file /home/jmrobles/Podcasts/Cowboys de Medianoche/cm251219_whisper_audio_en.html
Changing comment from